In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [30]:
# Cargar los archivos CSV
df_venta = pd.read_csv('venta_inmuebles_limpio.csv')
df_alquiler = pd.read_csv('alquiler_inmuebles_limpio.csv')

df_venta['Precio'] = df_venta['Precio'].astype(str).str.replace('.', '').astype(float)

# Verificar los nombres de las columnas en cada DataFrame
print("Columnas en df_venta:", df_venta.columns)
print("Columnas en df_alquiler:", df_alquiler.columns)



Columnas en df_venta: Index(['Enlace', 'Precio', 'Superficie Construida', 'Última Actualización',
       'Consumo Energético', 'Emisiones CO2', 'Tipo de operación',
       'Superficie construida:', 'Habitaciones:', 'Baños:', 'Conservación:',
       'Referencia:', 'Superficie útil:', 'Superficie solar:', 'Tipo de casa:',
       'Antigüedad:', 'Planta:', 'Gastos de comunidad:', 'Exterior:',
       'Interior:', 'descripcion'],
      dtype='object')
Columnas en df_alquiler: Index(['Descripción', 'Localización', 'Precio', 'Última Actualización',
       'Consumo Energético', 'Emisiones Co2', 'Tipo De Operación',
       'Superficie Construida', 'Superficie Útil', 'Habitaciones', 'Baños',
       'Antigüedad', 'Conservación', 'Planta', 'Tipo De Casa'],
      dtype='object')


In [31]:


# Fusionar los DataFrames usando una columna común como 'Localización' (ajusta si es necesario)
df_venta['Tipo de casa:'] = df_venta['Tipo de casa:'].str.strip().str.lower()
df_alquiler['Tipo De Casa'] = df_alquiler['Tipo De Casa'].str.strip().str.lower()

# Supongamos que la columna se llama 'Descripcion_venta' en df_venta y 'Descripcion_alquiler' en df_alquiler
df_merged = pd.merge(df_venta, df_alquiler, left_on='Tipo de casa:', right_on='Tipo De Casa', suffixes=('_compra', '_alquiler'))

# Calcular el ratio de precio de compra a precio de alquiler
df_merged['Ratio Compra/Alquiler'] = df_merged['Precio_compra'] / df_merged['Precio_alquiler']

# Eliminar filas con datos faltantes en el ratio
df_merged = df_merged.dropna(subset=['Ratio Compra/Alquiler'])

if 'Baños_compra' in df_merged.columns:
    df_merged['Baños'] = df_merged['Baños_compra']
elif 'Baños_alquiler' in df_merged.columns:
    df_merged['Baños'] = df_merged['Baños_alquiler']

# Segmentar el mercado según Localización, Tipo de Casa y Habitaciones
segmentacion = df_merged.groupby(['Tipo De Casa', 'Habitaciones', 'Baños']).agg({
    'Ratio Compra/Alquiler': 'mean',
    'Precio_compra': 'mean',
    'Precio_alquiler': 'mean'
}).reset_index()



# Ordenar para encontrar los segmentos con el mejor ratio
segmentacion = segmentacion.sort_values(by='Ratio Compra/Alquiler', ascending=False)

# Mostrar los segmentos con el mejor ratio de compra/alquiler
print("Top 10 segmentos con mejor ratio Compra/Alquiler")
print(segmentacion.head(10))

# Guardar los resultados en un archivo CSV
segmentacion.to_csv('segmentacion_mejor_ratio.csv', index=False)

Top 10 segmentos con mejor ratio Compra/Alquiler
   Tipo De Casa     Habitaciones            Baños  Ratio Compra/Alquiler  \
33         piso  No especificado  No especificado                    inf   
8          piso                3                2                    inf   
4          piso                2                2                    inf   
23         piso                5                6                    inf   
30         piso  No especificado                1              13.702212   
7          piso                3                1              12.368579   
3          piso                2                1              11.063482   
13         piso                4                1              10.982944   
0          piso                1                1               9.961217   
18         piso                4  No especificado               9.833808   

    Precio_compra  Precio_alquiler  
33   13521.485789      1077.727273  
8    13521.485789      2302.709497  
4  

In [32]:
# Verificar las columnas en el DataFrame fusionado
print("Columnas en df_merged:", df_merged.columns)

Columnas en df_merged: Index(['Enlace', 'Precio_compra', 'Superficie Construida_compra',
       'Última Actualización_compra', 'Consumo Energético_compra',
       'Emisiones CO2', 'Tipo de operación', 'Superficie construida:',
       'Habitaciones:', 'Baños:', 'Conservación:', 'Referencia:',
       'Superficie útil:', 'Superficie solar:', 'Tipo de casa:', 'Antigüedad:',
       'Planta:', 'Gastos de comunidad:', 'Exterior:', 'Interior:',
       'descripcion', 'Descripción', 'Localización', 'Precio_alquiler',
       'Última Actualización_alquiler', 'Consumo Energético_alquiler',
       'Emisiones Co2', 'Tipo De Operación', 'Superficie Construida_alquiler',
       'Superficie Útil', 'Habitaciones', 'Baños', 'Antigüedad',
       'Conservación', 'Planta', 'Tipo De Casa', 'Ratio Compra/Alquiler'],
      dtype='object')


In [33]:
import plotly.express as px

# Crear un gráfico de barras para mostrar el Ratio Compra/Alquiler por Tipo de Casa y Habitaciones
fig1 = px.bar(
    segmentacion,
    x='Tipo De Casa',
    y='Ratio Compra/Alquiler',
    color='Habitaciones',  
    title="Ratio Compra/Alquiler por Tipo de Casa y Número de Habitaciones",
    labels={'Ratio Compra/Alquiler': 'Ratio Compra/Alquiler promedio'},
    barmode='group'
)

fig1.show()

In [34]:
# Asegúrate de que la columna 'Habitaciones' sea de tipo numérico
segmentacion['Habitaciones'] = pd.to_numeric(segmentacion['Habitaciones'], errors='coerce')
segmentacion = segmentacion.dropna(subset=['Habitaciones'])

fig_3d = px.scatter_3d(
    segmentacion,
    x='Precio_compra',
    y='Precio_alquiler',
    z='Ratio Compra/Alquiler',
    color='Tipo De Casa',
    size='Habitaciones',  # Tamaño de los puntos según el número de habitaciones
    title="Relación entre Precio de Compra, Precio de Alquiler y Ratio Compra/Alquiler",
    labels={'Precio_compra': 'Precio de Compra', 'Precio_alquiler': 'Precio de Alquiler', 'Ratio Compra/Alquiler': 'Ratio Compra/Alquiler'}
)
fig_3d.show()

In [35]:
import plotly.graph_objects as go

fig_heatmap = go.Figure(data=go.Heatmap(
    z=segmentacion['Ratio Compra/Alquiler'],
    x=segmentacion['Precio_compra'],
    y=segmentacion['Precio_alquiler'],
    colorscale='Viridis'
))

fig_heatmap.update_layout(
    title="Mapa de Calor: Ratio Compra/Alquiler por Precio de Compra y Alquiler",
    xaxis_title="Precio de Compra",
    yaxis_title="Precio de Alquiler"
)
fig_heatmap.show()

In [36]:
fig_box = px.box(
    segmentacion,
    x='Tipo De Casa',
    y='Ratio Compra/Alquiler',
    title="Distribución del Ratio Compra/Alquiler por Tipo de Casa",
    labels={'Ratio Compra/Alquiler': 'Ratio Compra/Alquiler'}
)
fig_box.show()

In [37]:
fig_stacked_bar = px.bar(
    segmentacion,
    x='Tipo De Casa',
    y='Habitaciones',
    color='Baños',
    title="Distribución de Habitaciones y Baños por Tipo de Casa",
    labels={'Habitaciones': 'Número de Habitaciones', 'Baños': 'Número de Baños'},
    barmode='stack'
)
fig_stacked_bar.show()